<a href="https://colab.research.google.com/github/hamzaq453/Langchain-learnings/blob/main/Home_Finder_Agent_with_Command.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install -q -U langgraph langgraph-checkpoint-postgres psycopg psycopg-pool langchain_google_genai

In [42]:
# Step 2: Import necessary libraries and set up environment variables
from google.colab import userdata
import os

# Retrieve secrets
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
DB_URL = userdata.get('DB_URL')

# Set environment variables
os.environ["LANGCHAIN_API_KEY"] = GEMINI_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "real_estate_chatbot"

# Step 3: Initialize Database connection
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver

connection_kwargs = {"autocommit": True, "prepare_threshold": 0}
pool = ConnectionPool(conninfo=DB_URL, max_size=20, kwargs=connection_kwargs)

checkpointer = PostgresSaver(pool)
checkpointer.setup()

# Step 4: Import Chat Model and define state
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, RemoveMessage
from langgraph.graph import END
from langgraph.graph.state import CompiledStateGraph
from langgraph.types import Command
from langgraph.graph import StateGraph, START
from typing import List, Optional, TypedDict

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=GEMINI_API_KEY)

# Define the State class
class State(TypedDict):
    messages: List[HumanMessage | AIMessage]
    summary: Optional[str]
    homes_data: Optional[List[dict]]
    user_name: Optional[str]

# Step 5: Define Tools/Functions
# Tool 1: Answer questions using LLM
def answer_question(state: State):
    summary = state.get("summary", "")
    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]

    response = model.invoke(messages)
    return {"messages": response}

# Tool 2: Provide homes data (dummy for now)
def provide_homes_data(state: State):
    homes = [
        {"name": "Luxury Apartment", "price": "$1,200,000", "location": "Downtown, NY"},
        {"name": "Cozy Condo", "price": "$850,000", "location": "Brooklyn, NY"},
        {"name": "Family Home", "price": "$2,000,000", "location": "Queens, NY"},
        {"name": "Modern Studio", "price": "$750,000", "location": "Manhattan, NY"},
        {"name": "Penthouse Suite", "price": "$5,000,000", "location": "Upper East Side, NY"},
    ]
    # Generate LLM response summarizing the homes data
    prompt = (
        "Provide a user-friendly summary of the following homes data:\n\n"
        + "\n".join(
            [f"- {home['name']} priced at {home['price']} located in {home['location']}." for home in homes]
        )
    )
    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)
    return {"messages": response, "homes_data": homes}

# Step 6: Define command-based decision-making
def decide_next_tool(state: State) -> Command:
    messages = state.get("messages", [])
    last_message = messages[-1].content.lower() if messages else ""

    if "home" in last_message or "house" in last_message:
        goto = "provide_homes_data"
    else:
        goto = "answer_question"

    return Command(goto=goto)

# Step 7: Define summarization logic
def summarize_conversation(state: State):
    summary = state.get("summary", "")
    if summary:
        prompt = f"This is the summary so far: {summary}\n\nExtend the summary with new details above:"
    else:
        prompt = "Summarize the conversation above:"

    messages = state["messages"] + [HumanMessage(content=prompt)]
    response = model.invoke(messages)
    new_summary = response.content

    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": new_summary, "messages": delete_messages}

# Step 8: Build the chatbot state graph
builder = StateGraph(State)
builder.add_edge(START, "decide_next_tool")
builder.add_node(decide_next_tool)
builder.add_node(answer_question)
builder.add_node(provide_homes_data)
builder.add_node(summarize_conversation)

graph = builder.compile(checkpointer=checkpointer)

def pretty_print_response(output):
    print("\nChatbot Response:\n")
    # Ensure output['messages'] is iterable
    messages = output['messages'] if isinstance(output['messages'], list) else [output['messages']]
    for message in messages[-1:]:  # Only show the last AI response
        message.pretty_print()
    print("-" * 50)


# Step 10: Test the chatbot
# Initial state setup
initial_state = {
    "messages": [],
    "summary": None,
    "homes_data": None,
    "user_name": None,
}

# Thread configuration for the checkpointer
config = {"configurable": {"thread_id": "1"}}  # Ensure thread_id is provided

# Sample conversation
input_message = HumanMessage(content="Tell me about homes in New York")

# Merge initial state and input message using dictionary merging for compatibility
output = graph.invoke({**initial_state, "messages": [input_message]}, config)

# Pretty print the output
pretty_print_response(output)

# Check persisted state
print("\nPersisted State Metadata:")
graph_state = graph.get_state(config)
print(graph_state.metadata)



Chatbot Response:

================================== Ai Message ==================================

Looking for a home in New York City?  We have a variety of options:

* **Luxury Downtown:** A $1.2M apartment in the heart of Downtown Manhattan.
* **Brooklyn Charm:** A cozy condo in Brooklyn for $850,000.
* **Queens Family Home:** A spacious family home in Queens for $2M.
* **Manhattan Studio:** A stylish modern studio in Manhattan for $750,000.
* **Upscale Upper East Side:** A luxurious penthouse suite for $5M.

Prices range from $750,000 to $5,000,000, offering choices to suit different budgets and lifestyles.
--------------------------------------------------

Persisted State Metadata:
{'step': 57, 'source': 'loop', 'writes': {'provide_homes_data': {'messages': AIMessage(content='Looking for a home in New York City?  We have a variety of options:\n\n* **Luxury Downtown:** A $1.2M apartment in the heart of Downtown Manhattan.\n* **Brooklyn Charm:** A cozy condo in Brooklyn for $850